In [1]:
import numpy as np
from collections import Counter
import pandas as pd
from sklearn.metrics import confusion_matrix


## Pre-Processing

In [2]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
# encoder.fit_transform(array)

def prepro(data, isTest):
    
    # Filling empty numerical values with median
    data['Age'] = data['Age'].fillna(data['Age'].median())
    data['Fare'] = data['Fare'].fillna(data['Fare'].median())
    # Filling empty embarked with S
    data['Embarked'] = data['Embarked'].fillna('S')
    # Replacing categorical sex with integer values (0 for F and 1 for M)
    data['Sex'] = encoder.fit_transform(data['Sex'])  
    
    return data





In [3]:
dataset = pd.read_csv('titanic-train.csv')
#data_ = prepro(dataset,isTest)


X_train = dataset.iloc[:800,1:]
y_train = dataset.iloc[:800,0].values
X_test = dataset.iloc[800:892,1:]
y_test = dataset.iloc[800:892,0].values

X_test = prepro(X_test, False)
X_test = X_test.values
X_train = prepro(X_train, True)
X_train = X_train.values


## Implementing the decision tree

In [18]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [54]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=7):
        
        self.root = None
        
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self,X,y,depth=0):
        [number_of_samples,number_of_features] = X.shape
        if (number_of_samples > self.min_samples_split and depth <= self.max_depth):
            best_split = self.find_best_split(X,y,number_of_features)
            if best_split["info_gain"]>0:
                # build left
                left_subtree = self.build_tree(best_split["x_dataset_left"],best_split["y_dataset_left"],depth+1)
                # build right
                right_subtree = self.build_tree(best_split["x_dataset_right"],best_split["y_dataset_right"],depth+1)
               
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(y)
        
        return Node(value=leaf_value)
        

    def find_best_split(self,X,y,number_of_features):
        best_split = {}
        max_info_gain = -float("inf")
        count = 0
        for f_idx in range(number_of_features):
            feature_vals = X[:,f_idx]
            possible_thresholds = np.unique(feature_vals)
            for thr in possible_thresholds:
                gain = self.calculate_information_gain(feature_vals,y,thr)
                if gain>max_info_gain:
                    best_split["feature_index"] = f_idx
                    best_split["threshold"] = thr
                    left_idxs,right_idxs = self.split(feature_vals,thr)
                    best_split["y_dataset_left"],best_split["x_dataset_left"] = y[left_idxs],X[left_idxs,:]
                    best_split["y_dataset_right"],best_split["x_dataset_right"] = y[right_idxs],X[right_idxs,:]                    
                    best_split["info_gain"] = gain
                    max_info_gain = gain                
                
        return best_split

    def calculate_information_gain(self,X,y,thr):
        
        parent_entropy = self.calculate_entropy(y)
        left_idxs,right_idxs = self.split(X,thr)
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self.calculate_entropy(y[left_idxs]), self.calculate_entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the Information Gain
        information_gain = parent_entropy - child_entropy
        return information_gain


    def calculate_entropy(self,y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def split(self, X, split_thresh):
        left_idxs = np.argwhere(X <= split_thresh).flatten()
        right_idxs = np.argwhere(X > split_thresh).flatten()
        return left_idxs, right_idxs


    def calculate_leaf_value(self, Y):

        Y = list(Y)
        return max(Y, key=Y.count)


    def print_tree(self, tree=None, indent=" "):
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)


    def fit(self,X,y):
        self.root = self.build_tree(X,y,0)


    def predict(self, X):
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    

In [55]:
d = DecisionTreeClassifier()
d.fit(X_train,y_train)

In [56]:
d.print_tree()

X_2 <= 0 ? 0.1538543158678991
 left:X_0 <= 2 ? 0.1356548660743529
  left:X_6 <= 28.7125 ? 0.012893728904148666
    left:X_6 <= 27.75 ? 0.038599819629731025
        left:X_3 <= 23.0 ? 0.023445101512177935
                left:1
                right:X_3 <= 27.0 ? 0.05029145343797148
                                left:X_3 <= 24.0 ? 0.1239686396190054
                                                                left:X_6 <= 13.0 ? 0.2195121486796562
                                                                                                                                left:0
                                                                                                                                right:1
                                                                right:X_4 <= 0 ? 0.6365141682948128
                                                                                                                                left:1
                                       

In [57]:
Y_pred = d.predict(X_test) 
from sklearn.metrics import accuracy_score
print("The accuracy for k=7 : ",accuracy_score(y_test, Y_pred))

The accuracy for k=7 :  0.8461538461538461


In [58]:
print("The confusion matrix:")
confusion_matrix(y_test, Y_pred)

The confusion matrix:


array([[50,  7],
       [ 7, 27]], dtype=int64)

## Implementing a random forest

In [98]:
class forest():

    def __init__(self, n_trees=6, max_depth=5, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.n_features=n_feature
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeClassifier(max_depth=self.max_depth,min_samples_split=self.min_samples_split)
            X_sample, y_sample = self.random_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)


    def random_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def majority_voting(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self.majority_voting(pred) for pred in tree_preds])
        return predictions
        

In [99]:
f = forest(10,5,2)
f.fit(X_train,y_train)
y_pred = f.predict(X_test)

In [100]:
from sklearn.metrics import accuracy_score
print("The accuracy for 10 trees with max depth of 5 is: ",accuracy_score(y_test, y_pred))

The accuracy for 10 trees with max depth of 5 is:  0.8571428571428571


In [101]:
print("The confusion matrix:")
confusion_matrix(y_test, y_pred)

The confusion matrix:


array([[52,  5],
       [ 8, 26]], dtype=int64)